In [1]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string


In [2]:

labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]

batch_files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 
                'nogla0.csv','pew0.csv', 'rae0_0.csv', 
                 'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']

vid_files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "drdisrespect_FULL.csv"]
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

files = []
files.extend(vid_files)
files.extend(genre_files)
print(files)

['foxnews_FULL.csv', 'rae_FULL.csv', 'terror_FULL.csv', 'usanews.csv', 'pew0.csv', 'nogla0.csv', 'drdisrespect_FULL.csv', 'gaming_channels_GENRE.csv', 'news_channel_GENRE.csv']


# TOP BI-GRAMS BASED ON TFIDF SCORES

In [35]:
src_folder = "tfidf_scores_WORDS_per_video/"
dest_folder = "tfidf_scores_WORDS_per_video/"
top_dfs = []
for file in files:
    vals = {}
    c
    features = df.columns.difference(labels)
    dfs = []
    for label in labels:
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " words"
        dfs.append(dl)
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_WORDS_"+file)

,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,meltdown progress,2.309401,conservative tear,1.000000,beautiful kamala harris,0.447214,biden cheat,2.000000,traitor hang,1.506965,traitor hang,1.077060
1,magat meltdown,2.309401,fox total,1.000000,ivanka beautiful kamala,0.447214,gobble gobble,1.257449,civil war,0.860926,buck fiden,1.000000
2,magat meltdown progress,2.309401,sign traitor,1.000000,ivanka beautiful,0.447214,covid hoax,1.000000,need civil,0.601715,lose sucker,1.000000
3,maga tear,1.538530,jail right,1.000000,beautiful kamala,0.447214,hello dictatorship,1.000000,need civil war,0.601715,wait prison,1.000000
4,gobble gobble,1.517393,stimulus socialist,1.000000,kamala harris,0.447214,cheater bust,1.000000,hang gallow,0.595813,fk blm,1.000000
5,trump lol,1.474625,left garbage,1.000000,trump new,0.377964,raid server,1.000000,traitor hang treason,0.595813,fk acosta,1.000000
6,pardon turkey,1.425936,china virus,0.632233,big loser,0.377964,sign traitor,1.000000,hang treason,0.595813,pay mfer,1.000000
7,supreme court,1.302278,jail entire,0.577350,host big,0.377964,pay mfer,1.000000,traitor hang gallow,0.595813,swamp worm,1.000000
8,trump pardon,1.253666,entire left,0.577350,host big loser,0.377964,philadelphia rigger,1.000000,court traitor,0.475634,satan trump,1.000000
9,trump pardon turkey,1.084736,devil incarnate,0.577350,trump new host,0.377964,biden illuminati,1.000000,court traitor hang,0.475634,satan kuwait,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,bruh dude break,0.577350,00 hour,0.000000,toast thicc,0.447214,lesson old man,0.377964,00 hour,0.000000,00 hour,0.000000
1,bruh dude,0.577350,rae _raehype _raehype,0.000000,finally toast thicc,0.447214,teach lesson old,0.377964,rae _raehype _raehype,0.000000,rae _raehype _raehype,0.000000
2,dude break,0.577350,rae 2254,0.000000,toast thicc booty,0.447214,lesson old,0.377964,rae 2254,0.000000,rae 2254,0.000000
3,00 hour,0.000000,rae 2254 hug,0.000000,thicc booty,0.447214,time teach lesson,0.377964,rae 2254 hug,0.000000,rae 2254 hug,0.000000
4,rae _raehype,0.000000,rae 2nd,0.000000,finally toast,0.447214,time teach,0.377964,rae 2nd,0.000000,rae 2nd,0.000000
5,rae 2254 hug,0.000000,rae 2nd star,0.000000,rae 2nd,0.000000,teach lesson,0.377964,rae 2nd star,0.000000,rae 2nd star,0.000000
6,rae 2nd,0.000000,rae _rae100,0.000000,rae 2nd star,0.000000,old man,0.377964,rae _rae100,0.000000,rae _rae100,0.000000
7,rae 2nd star,0.000000,rae _raehype,0.000000,rae _rae100,0.000000,unlike video,0.301511,rae _raehype,0.000000,rae _raehype,0.000000
8,rae _rae100,0.000000,rae _raelove,0.000000,rae _raehype,0.000000,unsuscribe unlike,0.301511,rae _raelove,0.000000,rae _raelove,0.000000
9,rae _raelove,0.000000,pure support,0.000000,rae _raehype _raehype,0.000000,unlike video rae,0.301511,pure support,0.000000,pure support,0.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,die lmao,1.000000,black guy,2.852805,brian fan,1.000000,terroris simp,1.000000,kill nogla,1.000000,130 noob,1.000000
1,cory annoying,1.000000,guy nogla,1.340387,brian thicc,0.577350,dislike room,1.000000,burn wizard,1.000000,snitch fuck ridiculous,0.707107
2,lightning care,1.000000,guy nogla 2020,1.340387,brian thicc doe,0.577350,dislike trigger,1.000000,kill play,0.606875,snitch fuck,0.707107
3,sooo dirty,1.000000,nogla 2020,1.340387,thicc doe,0.577350,hope soon,0.577350,kill play guy,0.606875,snipe dickhead,0.627773
4,130 noob,1.000000,aussie weird,1.000000,release pleb,0.000000,trump2020 hope soon,0.577350,instant kill,0.577350,stream snipe dickhead,0.627773
5,breathes hes,0.707107,black guy nogla,0.927768,really sus,0.000000,trump2020 hope,0.577350,knife instant,0.577350,blocker fucker,0.622533
6,breathes hes impostor,0.707107,white guy nogla,0.480829,really think noah,0.000000,shoulda parent,0.447214,knife instant kill,0.577350,ad blocker fucker,0.622533
7,stream snipe dickhead,0.627773,white guy,0.422031,really want,0.000000,brown celebrity play,0.447214,play guy,0.513231,kill play,0.606875
8,snipe dickhead,0.627773,recently love terrorisor,0.000000,really want member,0.000000,orphan deserve shoulda,0.447214,leave open die,0.382034,kill play guy,0.606875
9,bad game watch,0.617562,recommendation midnight,0.000000,reason brian,0.000000,orphan deserve,0.447214,open die,0.382034,dang terroris,0.586032


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291875,china joe,12.082489,lady gag,3.000000,joe biden,9.277306,sunny yu,1.755135,biden suck,11.167500
1,joe biden,5.582116,vote trump,9.542417,lady gaga,1.918114,china joe,3.793307,beijing biden,1.574272,joe biden,7.951498
2,china joe,4.976935,biden suck,9.167500,commie bitch,1.000000,biden prison,3.663920,life matter,1.397841,lady gaga,5.049229
3,biden jail,4.881563,vote biden,7.403092,buck fiden,1.000000,biden china,3.000000,biden voter,1.111717,cnn suck,4.000000
4,sleepy joe,4.760010,biden prison,6.777448,make gag,0.899653,moron biden,2.852340,terror attack,1.076648,moron biden,3.469787
5,biden racist,4.519776,joe biden,5.888062,gaga make gag,0.899653,beijing biden,2.734768,trump hate,1.000000,biden supporter,3.447166
6,lady gaga,4.387135,china pay,5.086143,gag lady,0.899653,vote trump,2.712668,47 year,1.000000,biden harris,3.331422
7,biden prison,4.061779,china biden,4.716271,make gag lady,0.899653,biden corrupt,2.567092,chy na,1.000000,vote biden,3.328542
8,biden crime,3.905258,moron biden,4.704680,gaga make,0.873213,biden win,2.513682,trump china,1.000000,vote trump,2.848222
9,biden crime family,3.462861,biden 2020,4.682197,joe kameltoe,0.751828,fake news,2.454750,austria war,1.000000,china joe,2.808168


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,nancy cringe,1.000000,000 people,0.000000,felix gay,1.000000,members money,5.366563,000 people,0.000000,fake af,1.000000
1,nancy corpse,1.000000,response come,0.000000,require facetune picture,0.000000,charity members money,5.366563,response come,0.000000,try guy stfu,0.707107
2,nancy trash,1.000000,resize screen apparently,0.000000,resize screen apparently,0.000000,donations charity,5.366563,resize screen apparently,0.000000,try guy,0.707107
3,nancy boring,1.000000,respect btw,0.000000,respect btw,0.000000,donations charity members,5.366563,respect btw,0.000000,suck felix pick,0.577350
4,walk look like,0.424510,respect btw let,0.000000,respect btw let,0.000000,charity members,5.366563,respect btw let,0.000000,felix pick,0.577350
5,drunken man,0.424510,respond late,0.000000,respond late,0.000000,discord linked happened,0.638229,respond late,0.000000,suck felix,0.577350
6,way jack walk,0.424510,respond late everytime,0.000000,respond late everytime,0.000000,discord linked,0.638229,respond late everytime,0.000000,fck phasmo,0.333333
7,walk look,0.424510,respond ouija,0.000000,respond ouija,0.000000,join discord,0.430496,respond ouija,0.000000,fck phasmo audio,0.333333
8,way jack,0.393526,respond ouija board,0.000000,respond ouija board,0.000000,restart definitely,0.000000,respond ouija board,0.000000,lot time,0.333333
9,look like,0.352567,response come ask,0.000000,response come,0.000000,respond late,0.000000,response come ask,0.000000,audio lot time,0.333333


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,tom castillo,1.000000,blm joke,8.535057,sexy dick,1.000000,blm terrorist,1.237311,00 fucking,0.000000,pete fuckeddddd,1.000000
1,nogla racist,1.000000,death blm,3.503489,want dick,0.707107,covid fake,1.000000,speech hater,0.000000,shut bitch,1.000000
2,mom sus,1.000000,blm racist,3.000000,want dick jk,0.707107,purge 2021,1.000000,speech chat,0.000000,death blm,1.000000
3,speedy suck,1.000000,lives matter,1.705081,shaving ball hurt,0.585874,biden like,1.000000,speech chat fun,0.000000,code fckubtch,1.000000
4,mrtop5 gayyyyy,1.000000,george floyd,1.368968,shaving ball,0.585874,warming myth,0.722942,speech damit,0.000000,sexy dick,1.000000
5,richie 21 gay,0.707107,blm suck,1.322003,want suck,0.577350,voting joe know,0.707107,speech die,0.000000,chat retard,1.000000
6,richie 21,0.707107,blm terrorist,1.000000,want suck dick,0.577350,trump gae biden,0.707107,speech die shut,0.000000,white boy,0.828362
7,pete fuckin,0.707107,poc blm,1.000000,suck dick,0.577350,voting joe,0.707107,speech free,0.000000,pete fuckin idiot,0.707107
8,pete fuckin idiot,0.707107,hate jews,1.000000,ban nogla,0.559914,eat kid,0.707107,speech free spam,0.000000,pete fuckin,0.707107
9,deluxe fuck,0.600376,osers arginalized,1.000000,dick live,0.529994,eat kid peel,0.707107,speech hater block,0.000000,shut blm fucker,0.707107


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.309401,ni er,1.000000,baby meet,0.447214,000 000,0.000000,000 000,0.000000,non members,1.000000
1,people say,2.309401,000 000,0.000000,baby meet bedroom,0.447214,nice chat 3k,0.000000,nice chat 3k,0.000000,ew public chat,0.577350
2,say member,2.309401,nice chat 3k,0.000000,meet bedroom,0.447214,nice car,0.000000,nice car,0.000000,public chat,0.577350
3,member chat,1.374228,nice car,0.000000,hey baby,0.447214,nice 100 floor,0.000000,nice 100 floor,0.000000,ew public,0.577350
4,anti member,1.188723,nice 100 floor,0.000000,hey baby meet,0.447214,nice 100,0.000000,nice 100,0.000000,_flappin _flappin,0.475917
5,anti member chat,1.188723,nice 100,0.000000,000 000,0.000000,ni er,0.000000,ni er,0.000000,start play trash,0.336108
6,non members,1.000000,news chance free,0.000000,new song drop,0.000000,news chance free,0.000000,news chance free,0.000000,live 22 minute,0.336108
7,doc boomer,1.000000,news chance,0.000000,nice 100 floor,0.000000,news chance,0.000000,news chance,0.000000,live 22,0.336108
8,want member,0.611136,new transition doc,0.000000,nice 100,0.000000,new transition doc,0.000000,new transition doc,0.000000,22 minute,0.336108
9,bro bros trash,0.600742,new transition,0.000000,ni er,0.000000,new transition,0.000000,new transition,0.000000,minute start,0.336108


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.372892,blm joke,8.104467,ban nogla,1.000000,members money,5.366563,play guy,1.000000,white boy,1.566457
1,say member,2.351603,death blm,4.000000,live matter,1.000000,donations charity,5.366563,kill nogla,1.000000,stream snipe,1.000000
2,people say,2.199852,black guy,3.505535,felix gay,1.000000,charity members,5.366563,guy lol,1.000000,public chat,1.000000
3,member chat,1.264009,blm racist,3.000000,meet bedroom,1.000000,charity members money,5.366563,00 rae,0.000000,chat retard,1.000000
4,anti member chat,1.236869,lives matter,2.342428,00 rae,0.000000,donations charity members,5.366563,potato powers activate,0.000000,die lol,1.000000
5,anti member,1.236869,blm suck,2.000000,potato pick,0.000000,blm terrorist,1.483568,potato noise,0.000000,blm suck,1.000000
6,suck bad,1.000000,death black,1.489961,potato potato,0.000000,purge 2021,1.000000,potato pick,0.000000,non members,1.000000
7,bro bros,1.000000,george floyd,1.457013,potato potato family,0.000000,biden like,1.000000,potato potato,0.000000,ad blocker,1.000000
8,tom castillo,1.000000,guy nogla 2020,1.443434,potato potato potato,0.000000,ban nogla,1.000000,potato potato family,0.000000,death blm,1.000000
9,smell like,1.000000,guy nogla,1.411289,potato powers,0.000000,join discord,1.000000,potato potato potato,0.000000,guy stfu,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291081,china joe,12.101228,lady gag,3.000000,joe biden,10.618480,traitor hang,3.749889,biden suck,11.164323
1,joe biden,7.532736,vote trump,9.403320,lady gaga,1.919480,biden prison,3.807546,civil war,2.000000,joe biden,7.898715
2,sleepy joe,5.903194,biden suck,9.164323,make gag,1.007282,china joe,3.801986,sunny yu,1.754881,lady gaga,5.050634
3,china joe,4.986737,vote biden,7.472942,gag lady,1.007282,vote trump,3.723379,beijing biden,1.574307,cnn suck,4.000000
4,biden jail,4.882732,biden prison,6.890406,make gag lady,1.007282,biden win,3.014630,biden voter,1.465081,vote biden,3.875534
5,lady gaga,4.371027,joe biden,6.149795,commie bitch,1.000000,biden china,3.000000,life matter,1.398097,biden harris,3.518930
6,biden racist,4.315291,china pay,5.086512,kamala harris,1.000000,fake news,2.942018,terror attack,1.077138,moron biden,3.477117
7,biden prison,4.063162,china virus,4.946780,trump new,1.000000,moron biden,2.857837,mr president,1.000000,biden supporter,3.350534
8,biden crime,3.780495,biden 2020,4.941256,buck fiden,1.000000,biden harris,2.829684,say biden,1.000000,vote trump,2.965617
9,crime family,3.464373,china biden,4.716686,gaga make,0.977828,vote biden,2.787209,biden obama,1.000000,china joe,2.813161


# TOP BI-GRAMS BASED ON LOWEST P-VALUE

In [3]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder = "corr_words_per_label/"
corr_dfs = []
for file in files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    df.dropna(axis = 0, how = 'all', inplace = True)
    
    dic = {}
    dfs = {}
    #corr coeffcorr coeff
    for label in labels:
        dp = df[(df[label + " p-val"] > 0 )& ( df[label + " corr coeff"] > 0)].sort_values(by=[label + " p-val"], ascending = True).head(5)
        top_features = list(dp[label + " p-val"].index)
        #display(df[label + " p-val"])
        #dt = dp[ [label + " p-val", label + " corr coeff"]].T[top_features]
        dic[label] = top_features
   
    
    corr_coeff = {}
    for label in labels:
        col = ["top " + label]
        col.extend(dic[label])
        corr_df = pd.DataFrame(columns= col )
        
        for row in labels:
            new_row = {}
            for word in dic[label]:
                x = df.loc[word, row + " corr coeff"]
                row_label = "".join([w[0] for w in row.split()])
                
                if row == "Trolling":
                    row_label += "R"
                elif row == "Toxic":
                    row_label += "X"
                elif row == "Others":
                    row_label += "T" 
                
                new_row["top " + label] = row_label
                new_row[word] = x
            corr_df = corr_df.append(new_row, ignore_index=True)
        
        display(corr_df.set_index("top " + label).T.style.set_caption(file))
    

top Direct Harassment,DH,HS,SH,TR,OT,TX
bye bye,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
fox news,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
biden win,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
wear mask,0.774597,-0.309839,-0.309839,-0.309839,0.464758,-0.309839
president elect,0.774597,0.464758,-0.309839,-0.309839,-0.309839,-0.309839


top Hate Speech,DH,HS,SH,TR,OT,TX
country communist,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
lose lawsuit katie,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
manchurian candidate,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228


top Sexual Harassment,DH,HS,SH,TR,OT,TX
beautiful kamala,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
beautiful kamala harris,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
biden cheat,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden voter,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
talk gobbles,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon turkey pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon trump pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122


top Others,DH,HS,SH,TR,OT,TX
traitor hang,0.282843,-0.424264,-0.424264,-0.424264,0.707107,0.282843
civil war,-0.417365,0.139122,-0.417365,0.417365,0.695608,-0.417365
sock presses mouth,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228
war begin,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228
mr president,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228


...

top Hate Speech,DH,HS,SH,TR,OT,TX
kick joe,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
safe america,-0.309839,0.774597,-0.309839,-0.309839,-0.309839,0.464758
ruin california vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
ruin california,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
indianapolis countyworkers citizen,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839


top Sexual Harassment,DH,HS,SH,TR,OT,TX
make gag,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
make gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gaga make,0.417365,-0.417365,0.695608,-0.417365,-0.417365,0.139122
hell kameltoe,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
blm biden laptop,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
socialism communism,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
steal election,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden run,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden love,-0.309839,-0.309839,-0.309839,0.774597,-0.309839,0.464758


top Others,DH,HS,SH,TR,OT,TX
know talk,0.464758,-0.309839,-0.309839,-0.309839,0.774597,-0.309839
terror attack,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
dem want,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack vienna,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack austria,-0.424264,0.282843,-0.424264,0.282843,0.707107,-0.424264


top Toxic,DH,HS,SH,TR,OT,TX
žðÿ žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
bin biden crime,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black dementia,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597


# TOP POS SEQUENCE

In [167]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder =  "tfidf_scores_POS_per_video/"
dest_folder =  "0_FOR_ANALYSIS_RESULTS/"
for file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    dfs = []
    for label in labels: 
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " POS-SEQ"
        dfs.append(dl)
        
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_POS_TFIDF_"+file)

,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun,37.415141,noun,10.779411,adj,0.563302,noun,19.877071,noun,3.040468,noun,8.389635
1,verb,31.164422,verb,6.285894,propn adj propn,0.526614,propn,17.295058,verb,2.166718,propn,7.588780
2,propn,29.071201,propn,5.519395,adj propn propn,0.489947,verb,15.743957,noun verb,1.977364,verb,4.826851
3,noun noun,15.743240,adj,5.238574,adj noun,0.443087,propn propn,10.176826,verb noun,1.839372,propn propn,4.228124
4,verb noun,14.300662,noun verb,4.557598,adj noun adj,0.438233,verb noun,9.688992,noun verb noun,1.822580,noun noun,4.101740
5,adj,13.198396,noun noun,4.242164,propn,0.404483,adj,9.012993,adj,1.151808,verb noun,3.573496
6,propn propn,13.185006,adj noun,3.697568,noun adj noun,0.361946,noun verb,8.595650,verb adj,1.096270,noun verb,2.620820
7,noun verb,12.875164,verb noun,3.435631,adj propn,0.347468,adj noun,6.722191,adj noun,1.092112,adj noun,2.250257
8,adj noun,10.784184,verb adj,2.830533,propn adj,0.337164,propn verb,6.580229,verb noun propn,0.958597,adj,2.208073
9,propn verb,10.023063,verb propn,2.524286,verb adj noun,0.322816,noun noun,6.215255,verb adj noun,0.937751,emoji,2.121336


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun noun verb,0.642856,adj,0.000000,noun noun,0.434770,adj noun,0.407596,adj,0.000000,adj,0.000000
1,noun verb,0.452075,pron verb adv,0.000000,adv verb noun,0.432263,noun,0.389197,pron verb adv,0.000000,pron verb adv,0.000000
2,noun,0.417524,pron verb,0.000000,adv verb,0.363744,verb noun adj,0.386089,pron verb,0.000000,pron verb,0.000000
3,noun noun,0.376728,pron propn verb,0.000000,noun,0.361389,propn verb pron,0.383370,pron propn verb,0.000000,pron propn verb,0.000000
4,verb,0.257127,pron propn propn,0.000000,noun noun noun,0.339736,verb pron adv,0.355671,pron propn propn,0.000000,pron propn propn,0.000000
5,pron propn,0.000000,pron propn noun,0.000000,verb noun noun,0.325122,adj noun verb,0.349498,pron propn noun,0.000000,pron propn noun,0.000000
6,pron num,0.000000,pron propn emoji,0.000000,adv,0.259890,noun adj noun,0.323025,pron propn emoji,0.000000,pron propn emoji,0.000000
7,pron num noun,0.000000,pron propn adj,0.000000,verb noun,0.224790,adj,0.320747,pron propn adj,0.000000,pron propn adj,0.000000
8,pron pron,0.000000,pron propn,0.000000,verb,0.148371,propn noun propn,0.312129,pron propn,0.000000,pron propn,0.000000
9,pron pron verb,0.000000,pron pron verb,0.000000,pron propn adj,0.000000,noun propn verb,0.303686,pron pron verb,0.000000,pron pron verb,0.000000


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,adj,6.262154,propn,1.900911,noun,1.331537,noun,1.575940,verb,1.772870,noun,4.423290
1,noun,4.647241,propn propn,1.777734,propn noun,0.814670,verb,1.204777,verb propn,1.625556,verb,3.210137
2,propn,3.128368,propn propn num,1.523426,propn,0.699840,pron,1.079291,verb noun,0.931269,propn,2.863868
3,verb,3.027080,propn num,1.246450,noun noun,0.680212,adj noun,0.926392,propn,0.829306,noun verb,2.764093
4,noun verb,2.697634,num,1.186949,propn verb noun,0.651138,verb noun,0.901978,verb noun emoji,0.660798,verb noun,1.992662
5,noun adj,2.062227,propn propn propn,0.733211,propn verb,0.504707,noun verb verb,0.799679,verb noun verb,0.612388,adj,1.897318
6,noun verb noun,1.890778,adj noun,0.669173,verb noun,0.373706,verb verb,0.737525,noun,0.552222,noun verb noun,1.633606
7,adj noun,1.888751,noun,0.663005,verb,0.255712,pron noun pron,0.720609,noun emoji,0.517525,adj noun verb,1.362641
8,verb noun,1.780500,num adj noun,0.630892,pron num verb,0.000000,adj,0.705102,pron verb verb,0.494418,adj noun,1.310387
9,propn propn,1.721557,noun adj noun,0.603407,pron noun adj,0.000000,noun pron,0.699909,verb verb noun,0.459279,noun noun,1.295420


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn,100.972083,propn,179.180348,propn,6.359163,propn,93.222142,propn,16.659940,propn,91.692586
1,noun,78.507825,noun,161.349670,noun,4.741262,noun,80.537453,noun,12.665730,noun,87.625493
2,propn propn,58.113888,propn propn,103.537758,propn propn,4.079912,verb,55.383471,verb,10.486706,verb,53.825583
3,verb,47.432219,verb,90.582050,verb,3.428021,propn propn,49.804621,propn propn,9.573976,propn propn,49.399244
4,propn noun,34.989873,noun noun,66.551040,propn verb,2.520383,noun noun,33.236654,propn propn propn,6.643449,propn noun,36.108419
5,noun noun,31.607925,propn noun,64.137598,verb noun,2.385865,propn noun,32.818027,noun verb,5.914650,noun noun,33.759045
6,propn propn propn,27.810930,propn propn propn,57.325372,propn propn propn,2.161174,propn verb,30.841944,propn noun,4.560420,noun verb,29.090214
7,propn verb,26.942811,noun propn,52.554576,propn noun,2.116536,noun verb,28.275371,verb noun,4.535640,propn verb,28.323174
8,adj,24.208487,adj,51.166225,noun propn,1.922791,verb noun,27.519729,propn verb,4.390278,adj,25.977277
9,emoji,24.005195,propn verb,48.606222,noun noun,1.596570,adj,25.200321,noun propn,3.879007,propn propn propn,25.081894


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun,3.140246,adj,0.000000,propn adj,0.826709,propn,4.832405,adj,0.000000,noun,1.727996
1,noun noun,2.071747,pron noun propn,0.000000,adj,0.451423,noun propn noun,4.716698,pron noun propn,0.000000,propn,1.226991
2,noun verb,1.597451,pron verb,0.000000,propn,0.335813,verb noun propn,4.513010,pron verb,0.000000,adj,1.000000
3,verb,1.195154,pron propn verb,0.000000,pron num,0.000000,propn propn verb,4.315467,pron propn verb,0.000000,verb,0.873867
4,propn,1.121297,pron propn propn,0.000000,pron num noun,0.000000,propn verb noun,3.717106,pron propn propn,0.000000,noun verb,0.859421
5,verb verb adj,0.411287,pron propn pron,0.000000,pron pron,0.000000,noun propn,3.287750,pron propn pron,0.000000,noun verb propn,0.650557
6,propn verb verb,0.369422,pron propn noun,0.000000,pron pron adj,0.000000,propn verb,2.932063,pron propn noun,0.000000,verb noun verb,0.593971
7,noun propn verb,0.362706,pron propn emoji,0.000000,pron pron adv,0.000000,propn noun,2.755535,pron propn emoji,0.000000,verb propn,0.468838
8,verb adj noun,0.319955,pron propn,0.000000,pron pron verb,0.000000,propn propn,2.702094,pron propn,0.000000,propn verb pron,0.408974
9,noun propn,0.265276,pron pron verb,0.000000,pron verb adj,0.000000,noun,2.626304,pron pron verb,0.000000,noun noun,0.407246


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn,3.282023,noun,39.979560,noun,1.603356,noun,4.110639,adj,0.000000,noun,5.362787
1,noun,3.189209,propn verb,27.659010,verb,1.320928,verb,3.655919,pron adv verb,0.000000,propn,3.836415
2,propn verb,2.615165,propn,27.109501,verb verb,1.039740,propn,3.021545,pron emoji emoji,0.000000,verb,3.641114
3,verb,2.111612,verb,21.429544,propn,0.887493,verb noun,2.332825,pron emoji propn,0.000000,noun verb,2.659009
4,adj,1.998002,propn noun,11.010290,adj propn,0.853274,verb verb,2.068655,pron intj,0.000000,propn noun,2.586916
5,noun noun,1.920912,noun noun,8.060097,adj,0.789779,adj,2.022407,pron intj adj,0.000000,noun noun,2.469844
6,propn noun,1.262560,adj,5.077082,noun verb verb,0.747129,noun verb,1.997036,pron intj noun,0.000000,adj,2.109502
7,propn propn,1.213749,propn propn,4.902146,noun verb,0.611223,noun verb noun,1.494071,pron intj num,0.000000,propn propn,1.545498
8,verb adj,1.206330,noun verb,3.516448,verb verb noun,0.596675,noun propn,1.275659,pron noun,0.000000,verb noun,1.491839
9,propn verb adj,1.093966,propn adj,3.030817,pron,0.587252,propn adj,1.168937,pron noun adj,0.000000,adj adj,1.145716


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun,4.367875,adj,1.000000,intj noun verb,0.481892,adj,0.000000,adj,0.000000,noun verb,0.999604
1,emoji emoji,2.859338,pron propn pron,0.000000,verb pron noun,0.401418,pron propn pron,0.000000,pron propn pron,0.000000,verb,0.766379
2,emoji,2.800536,propn,0.000000,noun verb pron,0.381744,propn,0.000000,propn,0.000000,noun,0.727162
3,verb,2.774896,pron verb verb,0.000000,intj noun,0.315307,pron verb verb,0.000000,pron verb verb,0.000000,propn noun verb,0.679515
4,emoji emoji emoji,2.530647,pron verb propn,0.000000,pron noun,0.315307,pron verb propn,0.000000,pron verb propn,0.000000,emoji,0.582025
5,noun verb,2.480074,pron verb pron,0.000000,verb pron,0.288695,pron verb pron,0.000000,pron verb pron,0.000000,emoji emoji,0.524939
6,propn,2.435646,pron verb num,0.000000,noun verb,0.209522,pron verb num,0.000000,pron verb num,0.000000,verb noun,0.497602
7,propn noun verb,2.038545,pron verb noun,0.000000,pron,0.207741,pron verb noun,0.000000,pron verb noun,0.000000,propn,0.432621
8,propn noun,1.695424,pron verb emoji,0.000000,intj,0.197748,pron verb emoji,0.000000,pron verb emoji,0.000000,adj num noun,0.425209
9,verb noun,1.358510,pron verb adv,0.000000,noun,0.197313,pron verb adv,0.000000,pron verb adv,0.000000,propn noun,0.419703


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun,15.636480,noun,40.868284,noun,3.500575,noun,8.637986,verb,1.783968,noun,12.172266
1,propn,9.911171,propn verb,27.995858,propn,1.868646,propn,8.399532,verb propn,1.632589,verb,8.512259
2,verb,9.370500,propn,27.734072,verb,1.860601,verb,6.751179,verb noun,0.950298,propn,8.207529
3,adj,8.977922,verb,21.725637,noun noun,1.288422,verb noun,5.784096,propn,0.799496,noun verb,7.389451
4,noun verb,7.458495,propn noun,10.950675,adj,1.257017,verb noun propn,5.299617,verb noun emoji,0.652784,adj,5.327697
5,noun noun,5.398989,noun noun,8.084766,verb noun,1.127042,propn verb noun,4.950502,verb noun verb,0.612687,verb noun,4.387458
6,propn propn,4.238726,propn propn,6.638533,verb verb,0.994366,propn propn verb,4.767351,noun,0.529786,noun noun,4.293763
7,propn noun,4.137001,adj,6.605892,adj propn,0.849257,noun propn,4.691493,noun emoji,0.523773,propn noun,4.030487
8,propn verb,3.659408,noun verb,3.662675,noun verb,0.843867,noun propn noun,4.646405,pron verb verb,0.498560,noun verb noun,2.694919
9,adj noun,3.412918,propn propn propn,3.401211,propn adj,0.825241,propn verb,4.234867,verb verb noun,0.438078,adj noun,2.488886


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn,127.328142,propn,184.640885,propn,6.747159,propn,108.959819,propn,17.509670,propn,99.211311
1,noun,115.580045,noun,171.793639,noun,4.968938,noun,100.391514,noun,15.614501,noun,95.805910
2,verb,80.645714,propn propn,105.704980,propn propn,4.274895,verb,72.222830,verb,12.775223,verb,58.681463
3,propn propn,70.043865,verb,96.858247,verb,3.540764,propn propn,58.884801,propn propn,10.049174,propn propn,53.526768
4,noun noun,47.023963,noun noun,70.690420,propn verb,2.523303,noun noun,39.331314,noun verb,7.984031,propn noun,37.731913
5,propn noun,42.453242,propn noun,65.861141,verb noun,2.373236,propn noun,38.074264,propn propn propn,6.658013,noun noun,37.725530
6,adj,37.829103,propn propn propn,57.802739,propn propn propn,2.161768,verb noun,37.697882,verb noun,6.493883,noun verb,31.708023
7,propn verb,36.710438,adj,56.422644,propn noun,2.118816,noun verb,37.331476,propn noun,5.032415,propn verb,29.382483
8,noun verb,33.369821,noun propn,54.027091,noun propn,1.926598,propn verb,37.309750,noun propn,4.664395,adj,28.228263
9,verb noun,33.223370,propn verb,50.499723,noun noun,1.598252,adj,34.520225,noun verb noun,4.643711,propn propn propn,26.406266


# TOP POS SEQUENCE - P Value. Correlational Analysis

In [174]:
src_folder = "corr_pos_seq_per_label/"
dest_folder =  "0_FOR_ANALYSIS_RESULTS/"
corr_dfs = []
for file in files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    df.dropna(axis = 0, how = 'all', inplace = True)
    
    dic = {}
    dfs = {}
    #corr coeffcorr coeff
    for label in labels:
        dp = df[(df[label + " p-val"] > 0 )& ( df[label + " corr coeff"] > 0)].sort_values(by=[label + " p-val"], ascending = True).head(5)
        top_features = list(dp[label + " p-val"].index)
       
        #dt = dp[ [label + " p-val", label + " corr coeff"]].T[top_features]
        dic[label] = top_features
   
    
    corr_coeff = {}
    for label in labels:
        col = ["top " + label]
        col.extend(dic[label])
        corr_df = pd.DataFrame(columns= col )
        if label == "Toxic":
            display(dp[label + " p-val"].head(5))
        
        for row in labels:
            new_row = {}
            
            for word in dic[label]:
                x = df.loc[word, row + " corr coeff"]
                row_label = "".join([w[0] for w in row.split()])
                
                if row == "Trolling":
                    row_label += "R"
                elif row == "Toxic":
                    row_label += "X"
                elif row == "Others":
                    row_label += "T" 
                
                new_row["top " + label] = row_label
                new_row[word] = x
            corr_df = corr_df.append(new_row, ignore_index=True)
        
        display(corr_df.set_index("top " + label).T.style.set_caption(file))

top Direct Harassment,DH,HS,SH,TR,OT,TX
adv emoji,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
noun adj verb,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
adv verb verb,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
noun adv emoji,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
noun adv noun,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839


top Hate Speech,DH,HS,SH,TR,OT,TX
adj adj propn,0.464758,0.774597,-0.309839,-0.309839,-0.309839,-0.309839
adj emoji,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
propn intj,0.464758,0.774597,-0.309839,-0.309839,-0.309839,-0.309839
verb noun pron,0.464758,0.774597,-0.309839,-0.309839,-0.309839,-0.309839
noun noun num,0.141421,0.707107,-0.565685,-0.565685,0.141421,0.141421


top Sexual Harassment,DH,HS,SH,TR,OT,TX
adj noun adj,0.282843,-0.424264,0.282843,0.707107,-0.424264,-0.424264
adj propn propn,0.695608,-0.417365,0.139122,0.417365,-0.417365,-0.417365
propn adj propn,0.664211,-0.531369,0.132842,-0.132842,-0.531369,0.398527


top Trolling,DH,HS,SH,TR,OT,TX
intj noun,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
adv adj,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
verb noun emoji,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
adv verb noun,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
adj verb adj,0.000000,0.000000,-0.774597,0.774597,0.000000,0.000000


top Others,DH,HS,SH,TR,OT,TX
pron noun noun,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228
pron noun,0.664211,-0.531369,-0.531369,0.132842,0.398527,-0.132842
verb adj adj,-0.132842,0.132842,-0.531369,0.664211,0.398527,-0.531369
verb noun propn,0.664211,0.132842,-0.531369,-0.132842,0.398527,-0.531369
noun noun num,0.141421,0.707107,-0.565685,-0.565685,0.141421,0.141421


...

top Sexual Harassment,DH,HS,SH,TR,OT,TX
verb pron propn,-0.531369,0.664211,0.398527,-0.132842,-0.531369,0.132842
pron propn propn,-0.392792,0.130931,0.392792,-0.130931,-0.654654,0.654654
propn verb pron,-0.654654,0.654654,0.392792,0.130931,-0.392792,-0.130931


top Trolling,DH,HS,SH,TR,OT,TX
num num propn,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
adv verb adv,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
num noun num,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
adj verb adv,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
emoji propn num,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839


top Others,DH,HS,SH,TR,OT,TX
verb pron pron,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
pron pron,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
verb adv num,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
verb pron num,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228
pron num,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228


noun noun intj     0.070484
adj noun num       0.124880
intj propn verb    0.124880
adj propn adv      0.124880
emoji noun verb    0.124880
Name: Toxic p-val, dtype: float64

top Toxic,DH,HS,SH,TR,OT,TX
noun noun intj,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
adj noun num,-0.417365,0.417365,-0.417365,0.139122,-0.417365,0.695608
intj propn verb,-0.417365,0.417365,-0.417365,0.139122,-0.417365,0.695608
adj propn adv,0.417365,0.139122,-0.417365,-0.417365,-0.417365,0.695608
emoji noun verb,-0.417365,0.139122,-0.417365,0.417365,-0.417365,0.695608


In [169]:
src_folder = "../Annotations/CSVs/"
df = pd.read_csv(src_folder+"gaming_channels_GENRE.csv", index_col = 0)
df[df["Toxic"] == 1]["POS-SEQ"]

C:\Users\Phoebe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


4500                                   PRON VERB VERB NOUN
4651                                  NOUN VERB NOUN PROPN
4728                                       NOUN VERB PROPN
4750                                     NOUN ADJ VERB ADV
5192                                                 PROPN
5311                  PRON ADV VERB ADJ NOUN VERB ADJ NOUN
5361                                        NOUN VERB NOUN
5837                                   PROPN PRON VERB ADJ
6631                                         ADJ NOUN VERB
6647                                  PRON VERB PROPN VERB
6769                                   NOUN VERB VERB NOUN
6792                                            PROPN NOUN
7036                                    ADJ NOUN NOUN NOUN
7061                                        NOUN VERB NOUN
7172                                              NUM NOUN
7435                                     ADJ NOUN VERB ADJ
8561                                   PRON NOUN NOUN NO